In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ESRNN import ESRNN
plt.style.use('ggplot')
pd.options.display.max_rows = 999
np.set_printoptions(threshold=np.inf)

In [ ]:
# Plot
def plot_prediction(y, y_hat):
    n_y = len(y)
    n_yhat = len(y_hat)
    ds_y = np.array(range(n_y))
    ds_yhat = np.array(range(n_y-n_yhat, n_y))

    plt.plot(ds_y, y, label = 'y')
    plt.plot(ds_yhat, y_hat, label='y_hat')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def ffill_missing_dates_particular_serie(serie, min_date, max_date, freq):
    date_range = pd.date_range(start=min_date, end=max_date, freq=freq)
    unique_id = serie['unique_id'].unique()
    df_balanced = pd.DataFrame({'ds':date_range, 'key':[1]*len(date_range), 'unique_id': unique_id[0]})

    # Check balance
    check_balance = df_balanced.groupby(['unique_id']).size().reset_index(name='count')
    assert len(set(check_balance['count'].values)) <= 1
    df_balanced = df_balanced.merge(serie, how="left", on=['unique_id', 'ds'])

    df_balanced['y'] = df_balanced['y'].fillna(method='ffill')

    return df_balanced

def ffill_missing_dates_per_serie(df, freq, fixed_max_date=None):
    """Receives a DataFrame with a date column and forward fills the missing gaps in dates, not filling dates before
    the first appearance of a unique key

    Parameters
    ----------
    df: DataFrame
        Input DataFrame
    key: str or list
        Name(s) of the column(s) which make a unique time series
    date_col: str
        Name of the column that contains the time column
    freq: str
        Pandas time frequency standard strings, like "W-THU" or "D" or "M"
    numeric_to_fill: str or list
        Name(s) of the columns with numeric values to fill "fill_value" with
    """
    if fixed_max_date is None:
        df_max_min_dates = df[['unique_id', 'ds']].groupby('unique_id').agg(['min', 'max']).reset_index()
    else:
        df_max_min_dates = df[['unique_id', 'ds']].groupby('unique_id').agg(['min']).reset_index()
        df_max_min_dates['max'] = fixed_max_date

    df_max_min_dates.columns = df_max_min_dates.columns.droplevel()
    df_max_min_dates.columns = ['unique_id', 'min_date', 'max_date']

    df_list = []
    for index, row in df_max_min_dates.iterrows():
        df_id = df[df['unique_id'] == row['unique_id']]
        df_id = ffill_missing_dates_particular_serie(df_id, row['min_date'], row['max_date'], freq)
        df_list.append(df_id)

    df_dates = pd.concat(df_list).reset_index(drop=True).drop('key', axis=1)[['unique_id', 'ds', 'y']]

    return df_dates

# Train

In [ ]:
data = pd.read_csv('data/terra/train.csv')

In [ ]:
data = pd.read_csv('data/terra/train.csv')
data['ds'] = pd.to_datetime(data['day'], unit='d')
data['unique_id'] = data['cultivar'] + data['sitename']
data = data.rename(columns={'canopy_height':'y'})

#Series must be complete in the frequency
data = ffill_missing_dates_per_serie(data,'D')
data = data.drop_duplicates(['unique_id','ds'])

data.head()

In [ ]:
X_train = data[['unique_id','ds']]
X_train['x'] = '1'
y_train = data[['unique_id','ds','y']]

In [ ]:
data_plot = y_train[y_train['unique_id']=='PI157030MAC Field Scanner Season 4 Range 17 Column 14']
plt.plot(data_plot['ds'], data_plot['y'], label = 'y')
plt.show()

# Test

In [ ]:
data_test = pd.read_csv('data/terra/test_real.csv')
data_test['ds'] = pd.to_datetime(data_test['day'], unit='d')
data_test['unique_id'] = data_test['cultivar'] + data_test['sitename']
data_test.head()

In [ ]:
X_test = data_test[['unique_id','ds','canopy_height']]
X_test.columns = ['unique_id', 'ds', 'y']
uniques = X_test['unique_id'].unique()

In [ ]:
# Train model
esrnn = ESRNN(max_epochs=1, batch_size=16, learning_rate=3e-4, gradient_clipping_threshold=20,
              dilations=[[1, 7], [28]], add_nl_layer=True, per_series_lr_multip=1.0, 
              seasonality=7, input_size=7, output_size=50, max_periods=20, level_variability_penalty=80,
              rnn_weight_decay=0)
esrnn.fit(X_train, y_train, random_seed=1)

In [ ]:
y_hat = esrnn.predict(X_test[['unique_id']])
X_plot = y_train.append(y_hat)
plot_id = 0
y_test_plot = X_plot.loc[X_plot['unique_id']==uniques[plot_id]]
plot_prediction(y_test_plot['y'], y_test_plot['y_hat'])

In [ ]:
y_hat = esrnn.predict(X_test[['unique_id','ds','y']])

In [ ]:
np.abs(y_hat['y_hat']-y_hat['y']).mean()

# Sigmoidal fit

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
data = pd.read_csv('data/terra/train.csv')
data['unique_id'] = data['cultivar'] + data['sitename']
data = data.rename(columns={'canopy_height':'y', 'day':'t'})
data = data.drop_duplicates(['unique_id','t'])
X_train = data[['unique_id','t', 'y']]
X_train.head()

In [ ]:
data_test = pd.read_csv('data/terra/test_real.csv')
data_test['unique_id'] = data_test['cultivar'] + data_test['sitename']
data_test = data_test.rename(columns={'canopy_height':'y', 'day':'t'})
X_test = data_test[['unique_id','t','y']]
X_test.head()

In [ ]:
class logistic_function(object):
    def __init__(self, bounds):
        self.bounds = bounds
        
    def logisticf(self, x, a, b, c):
        return  a / (1.0 + np.exp(-b*(x-c)))
        
    def train(self, X_df):
        def logisticf(x, a, b, c):
            return a / (1.0 + np.exp(-b*(x-c)))
        
        for i, serie in enumerate(self.unique_ids):
            x = X_df[X_df['unique_id']==serie]['t']
            y = X_df[X_df['unique_id']==serie]['y']
    
            popt, _ = curve_fit(logisticf, x, y, bounds=self.bounds)
            self.parameters[i,0] = popt[0]
            self.parameters[i,1] = popt[1]
            self.parameters[i,2] = popt[2]
            self.last_day[i] = max(x)
        
    def fit(self, X_df):
        X_df = X_df.sort_values(['unique_id','t'])

        self.unique_ids = X_df['unique_id'].unique()
        self.n_series = len(self.unique_ids)
        self.parameters = np.zeros((self.n_series, 3))
        self.last_day = np.zeros(self.n_series)
        
        # Train
        self.train(X_df)
    
    def predict(self, X_df, predict_train = False):
        X_df = X_df.sort_values(['unique_id','t'])
        max_date = X_df['t'].max() + 1
        test_ids = X_df['unique_id'].unique()
        
        Y_hat_panel = pd.DataFrame(columns=['unique_id', 't', 'y_hat'])
        
        for i, unique_id in enumerate(test_ids):
            Y_hat_id = pd.DataFrame(np.zeros(shape=(int(max_date), 1)), columns=["y_hat"]) # -self.last_day[i]
                
            t_test = np.arange(0, max_date, 1) #self.last_day[i]
            y_hat = self.logisticf(t_test, self.parameters[i,0], self.parameters[i,1], self.parameters[i,2])
            Y_hat_id.iloc[:, 0] = y_hat
            Y_hat_id["unique_id"] = unique_id
            Y_hat_id["t"] = t_test
            Y_hat_panel = Y_hat_panel.append(Y_hat_id, sort=False).reset_index(drop=True)
        
        return Y_hat_panel

In [ ]:
lf = logistic_function(([250, 0, 0], [400, 2, 120]))

In [ ]:
lf.fit(X_train)

In [ ]:
preds = lf.predict(X_test)

In [ ]:
y_hat = X_test.merge(preds,on=['unique_id','t'], how='left')

In [ ]:
np.abs(y_hat['y_hat']-y_hat['y']).mean()

In [ ]:
X_plot = X_train.merge(preds,on=['unique_id','t'], how='outer')
X_plot = X_plot.merge(X_test,on=['unique_id','t'], how='outer')

In [ ]:
ids = 30
x_plot = X_plot[X_plot['unique_id']==uniques[ids]].sort_values('t')
x = x_plot['t'].values
y_train = x_plot['y_x'].values
y_test = x_plot['y_y'].values
y_hat = x_plot['y_hat'].values

In [ ]:
plt.plot(x, y_hat, label = 'prediction')
plt.scatter(x, y_train, label = 'train', c ='black')
plt.scatter(x, y_test, label = 'test', c ='blue')
plt.legend(loc='upper left')
plt.show()

In [ ]:
test_crops = X_plot.copy()
test_crops['error'] = test_crops['y_y'] - test_crops['y_hat']
test_crops = test_crops[test_crops['error'].notnull()]
test_crops = test_crops[['unique_id','t','y_y','y_hat','error']]

In [ ]:
test_crops = test_crops.merge(data_test[['unique_id','t','cultivar','sitename']],on=['unique_id','t'])

In [ ]:
test_crops[['cultivar','error']].groupby('cultivar')['error'].mean().reset_index()